# 02 Clean and Engineer
Load raw scrape CSVs, deduplicate, parse fields, derive flags, and write `data/processed/processed.csv`.

### Setup paths and imports
we load the data and make sure the cleaning and combining functions are in place

In [ ]:
import sys
from pathlib import Path
import pandas as pd

ROOT = Path.cwd()
if ROOT.name == 'notebooks':
    ROOT = ROOT.parent
sys.path.append(str(ROOT))

from src.processing.combine import list_raw_paths, load_and_concat, drop_exact_duplicates
from src.processing.cleaning import (
    parse_price,
    parse_area,
    parse_floor,
    parse_max_floor,
    derive_floor_flags,
    map_heating,
    map_construction,
    derive_newbuild,
    standardize_district,
)


### Configure raw input paths

In [ ]:
RAW_GLOB = str(ROOT / 'data/raw/sales/raw_*_pilot.csv')
raw_paths = list_raw_paths(RAW_GLOB)
raw_paths

[PosixPath('/home/gogo/dev/sofia-real-estate-risk-map/data/raw/sales/raw_room1_pilot.csv'),
 PosixPath('/home/gogo/dev/sofia-real-estate-risk-map/data/raw/sales/raw_room2_pilot.csv'),
 PosixPath('/home/gogo/dev/sofia-real-estate-risk-map/data/raw/sales/raw_room3_pilot.csv')]

### Load, concatenate, drop duplicates

In [ ]:
df_raw = load_and_concat(raw_paths)
df_raw = drop_exact_duplicates(df_raw)
df_raw.head()

,url,listing_id,source,price_raw,area_raw,rooms,district_raw,floor_raw,max_floor_raw,heat_raw,construction_raw,year_raw,desc_text
0,https://www.imot.bg/obiava-1a176060746046054-p...,ida1a176060746046054,imot.bg,115 000 €,55 m 2,1,"град София, Овча купел 2",6.0,10.0,ТEЦ: ДА,Тухла,2027.0,Описание на имота: Едностаен Апартамент 'На Зе...
1,https://www.imot.bg/obiava-1a176352251944193-p...,ida1a176352251944193,imot.bg,125 000 €,44 m 2,1,"град София, Банишора",12.0,14.0,Газ: НЕ; ТEЦ: ДА,ЕПК,1980.0,Описание на имота: Продава се едностаен апарта...
2,https://www.imot.bg/obiava-1a176276279674731-p...,ida1a176276279674731,imot.bg,163 000 €,50 m 2,1,"град София, Борово",6.0,6.0,ТEЦ: Лок.отопл.,Тухла,2005.0,Описание на имота: Обзаведено жилище в съвреме...
3,https://www.imot.bg/obiava-1a176183615194066-p...,ida1a176183615194066,imot.bg,98 200 €,42 m 2,1,"град София, Витоша",3.0,4.0,Газ: НЕ; ТEЦ: НЕ,Тухла,2026.0,Описание на имота: Предлагаме ви стилно жилище...
4,https://www.imot.bg/obiava-1a176170780994189-p...,ida1a176170780994189,imot.bg,103 000 €,47 m 2,1,"град София, Витоша",6.0,10.0,Газ: НЕ,Тухла,2026.0,Описание на имота: До бъдещата метростанция на...


### Parse numeric and categorical fields

In [ ]:
EUR_TO_BGN = 1.95583
df = df_raw.copy()
df['price_eur'] = parse_price(df['price_raw'])
df['price_bgn'] = df['price_eur'] * EUR_TO_BGN
df['area_m2'] = parse_area(df['area_raw'])
df['floor'] = parse_floor(df['floor_raw'])
df['max_floor'] = parse_max_floor(df['max_floor_raw'])
df['heat'] = map_heating(df['heat_raw'])
df['construction_type'] = map_construction(df['construction_raw'])
df['newbuild'] = derive_newbuild(df['year_raw'], df['construction_raw'], df.get('desc_text'))
df['district'] = standardize_district(df['district_raw'])
df = derive_floor_flags(df)
df.head()


,url,listing_id,source,price_raw,area_raw,rooms,district_raw,floor_raw,max_floor_raw,heat_raw,...,price_eur,area_m2,floor,max_floor,heat,construction_type,newbuild,district,is_ground_floor,is_top_floor
0,https://www.imot.bg/obiava-1a176060746046054-p...,ida1a176060746046054,imot.bg,115 000 €,55 m 2,1,"град София, Овча купел 2",6.0,10.0,ТEЦ: ДА,...,115000.0,55.0,6.0,10.0,other,brick,1.0,Овча купел 2,0,0
1,https://www.imot.bg/obiava-1a176352251944193-p...,ida1a176352251944193,imot.bg,125 000 €,44 m 2,1,"град София, Банишора",12.0,14.0,Газ: НЕ; ТEЦ: ДА,...,125000.0,44.0,12.0,14.0,gas,epk,0.0,Банишора,0,0
2,https://www.imot.bg/obiava-1a176276279674731-p...,ida1a176276279674731,imot.bg,163 000 €,50 m 2,1,"град София, Борово",6.0,6.0,ТEЦ: Лок.отопл.,...,163000.0,50.0,6.0,6.0,other,brick,0.0,Борово,0,1
3,https://www.imot.bg/obiava-1a176183615194066-p...,ida1a176183615194066,imot.bg,98 200 €,42 m 2,1,"град София, Витоша",3.0,4.0,Газ: НЕ; ТEЦ: НЕ,...,98200.0,42.0,3.0,4.0,gas,brick,1.0,Витоша,0,0
4,https://www.imot.bg/obiava-1a176170780994189-p...,ida1a176170780994189,imot.bg,103 000 €,47 m 2,1,"град София, Витоша",6.0,10.0,Газ: НЕ,...,103000.0,47.0,6.0,10.0,gas,brick,1.0,Витоша,0,0


### Keep relevant columns for modeling

In [ ]:
cols = [
    'url', 'listing_id', 'price_bgn', 'price_eur', 'area_m2', 'rooms', 'floor', 'max_floor',
    'is_ground_floor', 'is_top_floor', 'heat', 'construction_type', 'newbuild', 'district'
]
df_model = df[cols].copy()
df_model.head()

,url,listing_id,price_bgn,price_eur,area_m2,rooms,floor,max_floor,is_ground_floor,is_top_floor,heat,construction_type,newbuild,district
0,https://www.imot.bg/obiava-1a176060746046054-p...,ida1a176060746046054,224920.450,115000.0,55.0,1,6.0,10.0,0,0,other,brick,1.0,Овча купел 2
1,https://www.imot.bg/obiava-1a176352251944193-p...,ida1a176352251944193,244478.750,125000.0,44.0,1,12.0,14.0,0,0,gas,epk,0.0,Банишора
2,https://www.imot.bg/obiava-1a176276279674731-p...,ida1a176276279674731,318800.290,163000.0,50.0,1,6.0,6.0,0,1,other,brick,0.0,Борово
3,https://www.imot.bg/obiava-1a176183615194066-p...,ida1a176183615194066,192062.506,98200.0,42.0,1,3.0,4.0,0,0,gas,brick,1.0,Витоша
4,https://www.imot.bg/obiava-1a176170780994189-p...,ida1a176170780994189,201450.490,103000.0,47.0,1,6.0,10.0,0,0,gas,brick,1.0,Витоша


### Save processed data

In [ ]:
processed_path = ROOT / 'data/processed/processed.csv'
processed_path.parent.mkdir(parents=True, exist_ok=True)
df_model.to_csv(processed_path, index=False)
processed_path

PosixPath('/home/gogo/dev/sofia-real-estate-risk-map/data/processed/processed.csv')